In [1]:
import athena_timeseries
import boto3
import numpy as np
import pandas as pd

In [2]:
boto3_session = boto3.Session(region_name="ap-northeast-1")

tsdb = athena_timeseries.AthenaTimeSeries(
    boto3_session=boto3_session,
    glue_db_name="jquants_api",
    s3_path="s3://japanese-stocks/jquants-api",
)

# TOPIX

In [10]:
# TOPIXのヒストリカルデータ
df = pd.read_pickle("../colab_data/topix_20080101_20240406.pkl")
df["Date"] = pd.to_datetime(df["Date"])

# symbol represent a group of data for given data columns
df["symbol"] = "jquants_api"

# timestamp should be UTC timezone but without tz info
df["dt"] = df["Date"].dt.tz_localize(None)

# partition_dt must be date, data will be updated partition by partition with use of this column.
# Every time, you have to upload all the data for a given partition_dt, otherwise older will be gone.
df["partition_dt"] = df["dt"].dt.date.map(lambda x: x.replace(day=1))

display(df.head())
df.info()

,Date,Open,High,Low,Close,symbol,dt,partition_dt
0,2008-05-07,1390.16,1400.65,1384.97,1393.28,jquants_api,2008-05-07,2008-05-01
1,2008-05-08,1384.06,1385.75,1372.95,1372.95,jquants_api,2008-05-08,2008-05-01
2,2008-05-09,1372.40,1373.73,1340.79,1341.76,jquants_api,2008-05-09,2008-05-01
3,2008-05-12,1331.49,1345.48,1327.21,1342.79,jquants_api,2008-05-12,2008-05-01
4,2008-05-13,1351.17,1364.29,1344.34,1360.05,jquants_api,2008-05-13,2008-05-01


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3896 entries, 0 to 3895
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   Date          3896 non-null   datetime64[ns]
 1   Open          3896 non-null   float64       
 2   High          3896 non-null   float64       
 3   Low           3896 non-null   float64       
 4   Close         3896 non-null   float64       
 5   symbol        3896 non-null   object        
 6   dt            3896 non-null   datetime64[ns]
 7   partition_dt  3896 non-null   object        
dtypes: datetime64[ns](2), float64(4), object(2)
memory usage: 243.6+ KB


In [11]:
tsdb.upload(table_name="topix", df=df)

{'paths': ['s3://japanese-stocks/jquants-api/topix/partition_dt=2017-12-01/symbol=jquants_api/8b1ea938e3104c3381366b2a6b4f1daf.snappy.parquet',
  's3://japanese-stocks/jquants-api/topix/partition_dt=2010-08-01/symbol=jquants_api/8b1ea938e3104c3381366b2a6b4f1daf.snappy.parquet',
  's3://japanese-stocks/jquants-api/topix/partition_dt=2008-07-01/symbol=jquants_api/8b1ea938e3104c3381366b2a6b4f1daf.snappy.parquet',
  's3://japanese-stocks/jquants-api/topix/partition_dt=2020-05-01/symbol=jquants_api/8b1ea938e3104c3381366b2a6b4f1daf.snappy.parquet',
  's3://japanese-stocks/jquants-api/topix/partition_dt=2024-02-01/symbol=jquants_api/8b1ea938e3104c3381366b2a6b4f1daf.snappy.parquet',
  's3://japanese-stocks/jquants-api/topix/partition_dt=2020-11-01/symbol=jquants_api/8b1ea938e3104c3381366b2a6b4f1daf.snappy.parquet',
  's3://japanese-stocks/jquants-api/topix/partition_dt=2021-08-01/symbol=jquants_api/8b1ea938e3104c3381366b2a6b4f1daf.snappy.parquet',
  's3://japanese-stocks/jquants-api/topix/part

In [3]:
# 試しにデータ抽出
# raw_close = tsdb.query(
#     table_name='topix',
#     field='date,open,high,low',
#     start_dt='2024-02-01 00:00:00', # yyyy-mm-dd HH:MM:SS, inclusive
#     end_dt='2024-04-10 23:59:59', # yyyy-mm-dd HH:MM:SS, inclusive
#     symbols=['jquants_api'],
# )
# raw_close

(44, 6)
        date     open     high      low       symbol         dt
0 2024-04-01  2782.19  2784.30  2713.14  jquants_api 2024-04-01
1 2024-04-02  2726.61  2742.74  2705.47  jquants_api 2024-04-02
2 2024-04-03  2697.47  2715.99  2686.79  jquants_api 2024-04-03
3 2024-04-04  2731.50  2752.21  2728.14  jquants_api 2024-04-04
4 2024-04-05  2699.53  2706.08  2679.73  jquants_api 2024-04-05


,date,open,high,low,symbol,dt
0,2024-02-01,2533.10,2543.88,2527.17,jquants_api,2024-02-01
1,2024-02-02,2543.89,2550.29,2530.96,jquants_api,2024-02-02
2,2024-02-05,2557.47,2561.29,2546.56,jquants_api,2024-02-05
3,2024-02-06,2547.22,2548.95,2532.53,jquants_api,2024-02-06
4,2024-02-07,2536.51,2558.10,2534.50,jquants_api,2024-02-07
5,2024-02-08,2557.35,2572.41,2542.72,jquants_api,2024-02-08
6,2024-02-09,2560.96,2576.38,2552.09,jquants_api,2024-02-09
7,2024-02-13,2579.33,2613.94,2577.23,jquants_api,2024-02-13
8,2024-02-14,2598.04,2598.49,2573.70,jquants_api,2024-02-14
9,2024-02-15,2598.88,2599.72,2581.87,jquants_api,2024-02-15


# 株価

In [5]:
# TOPIXのヒストリカルデータ
df = pd.read_pickle("../colab_data/price_20080101_20240407.pkl")
df["Date"] = pd.to_datetime(df["Date"])

# symbol represent a group of data for given data columns
df["symbol"] = "jquants_api"

# timestamp should be UTC timezone but without tz info
df["dt"] = df["Date"].dt.tz_localize(None)

# partition_dt must be date, data will be updated partition by partition with use of this column.
# Every time, you have to upload all the data for a given partition_dt, otherwise older will be gone.
df["partition_dt"] = df["dt"].dt.date.map(lambda x: x.replace(day=1))

print(df.shape)
df.head()

(13507205, 44)


,Date,Code,Open,High,Low,Close,UpperLimit,LowerLimit,Volume,TurnoverValue,...,AfternoonLowerLimit,AfternoonVolume,AfternoonAdjustmentOpen,AfternoonAdjustmentHigh,AfternoonAdjustmentLow,AfternoonAdjustmentClose,AfternoonAdjustmentVolume,symbol,dt,partition_dt
0,2008-05-07,13010,173.0,177.0,173.0,176.0,0,0,302000.0,5.291200e+07,...,0,131000.0,1770.0,1770.0,1750.0,1760.0,13100.0,jquants_api,2008-05-07,2008-05-01
1,2008-05-07,77330,3630.0,3630.0,3540.0,3560.0,0,0,1837000.0,6.556140e+09,...,0,926000.0,890.0,895.0,885.0,890.0,3704000.0,jquants_api,2008-05-07,2008-05-01
2,2008-05-07,77340,739.0,744.0,729.0,740.0,0,0,13800.0,1.023440e+07,...,0,9000.0,371.5,372.0,370.0,370.0,18000.0,jquants_api,2008-05-07,2008-05-01
3,2008-05-07,77350,470.0,478.0,468.0,477.0,0,0,2305000.0,1.094041e+09,...,0,1555000.0,1192.5,1195.0,1180.0,1192.5,622000.0,jquants_api,2008-05-07,2008-05-01
4,2008-05-07,77360,22.0,23.0,21.0,23.0,0,0,399200.0,8.777700e+06,...,0,205200.0,22.0,23.0,21.0,23.0,205200.0,jquants_api,2008-05-07,2008-05-01


In [6]:
df.iloc[:, :20].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13507205 entries, 0 to 13507204
Data columns (total 20 columns):
 #   Column            Dtype         
---  ------            -----         
 0   Date              datetime64[ns]
 1   Code              object        
 2   Open              float32       
 3   High              float32       
 4   Low               float32       
 5   Close             float32       
 6   UpperLimit        int8          
 7   LowerLimit        int8          
 8   Volume            float32       
 9   TurnoverValue     float32       
 10  AdjustmentFactor  float32       
 11  AdjustmentOpen    float32       
 12  AdjustmentHigh    float32       
 13  AdjustmentLow     float32       
 14  AdjustmentClose   float32       
 15  AdjustmentVolume  float32       
 16  MorningOpen       float32       
 17  MorningHigh       float32       
 18  MorningLow        float32       
 19  MorningClose      float32       
dtypes: datetime64[ns](1), float32(16), int8(2), 

In [8]:
df.iloc[:, 20:40].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13507205 entries, 0 to 13507204
Data columns (total 20 columns):
 #   Column                    Dtype  
---  ------                    -----  
 0   MorningUpperLimit         int8   
 1   MorningLowerLimit         int8   
 2   MorningVolume             float32
 3   MorningTurnoverValue      float32
 4   MorningAdjustmentOpen     float32
 5   MorningAdjustmentHigh     float32
 6   MorningAdjustmentLow      float32
 7   MorningAdjustmentClose    float32
 8   MorningAdjustmentVolume   float32
 9   AfternoonOpen             float32
 10  AfternoonHigh             float32
 11  AfternoonLow              float32
 12  AfternoonClose            float32
 13  AfternoonUpperLimit       int8   
 14  AfternoonLowerLimit       int8   
 15  AfternoonVolume           float32
 16  AfternoonAdjustmentOpen   float32
 17  AfternoonAdjustmentHigh   float32
 18  AfternoonAdjustmentLow    float32
 19  AfternoonAdjustmentClose  float32
dtypes: float32(16), int8(4

In [9]:
df.iloc[:, 40:].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13507205 entries, 0 to 13507204
Data columns (total 4 columns):
 #   Column                     Dtype         
---  ------                     -----         
 0   AfternoonAdjustmentVolume  float32       
 1   symbol                     object        
 2   dt                         datetime64[ns]
 3   partition_dt               object        
dtypes: datetime64[ns](1), float32(1), object(2)
memory usage: 360.7+ MB


In [10]:
tsdb.upload(table_name="price", df=df)

{'paths': ['s3://japanese-stocks/jquants-api/price/partition_dt=2009-11-01/symbol=jquants_api/3726870b5a5d4b9ca4b2308d03d1a620.snappy.parquet',
  's3://japanese-stocks/jquants-api/price/partition_dt=2018-11-01/symbol=jquants_api/3726870b5a5d4b9ca4b2308d03d1a620.snappy.parquet',
  's3://japanese-stocks/jquants-api/price/partition_dt=2010-09-01/symbol=jquants_api/3726870b5a5d4b9ca4b2308d03d1a620.snappy.parquet',
  's3://japanese-stocks/jquants-api/price/partition_dt=2017-07-01/symbol=jquants_api/3726870b5a5d4b9ca4b2308d03d1a620.snappy.parquet',
  's3://japanese-stocks/jquants-api/price/partition_dt=2015-02-01/symbol=jquants_api/3726870b5a5d4b9ca4b2308d03d1a620.snappy.parquet',
  's3://japanese-stocks/jquants-api/price/partition_dt=2016-07-01/symbol=jquants_api/3726870b5a5d4b9ca4b2308d03d1a620.snappy.parquet',
  's3://japanese-stocks/jquants-api/price/partition_dt=2014-04-01/symbol=jquants_api/3726870b5a5d4b9ca4b2308d03d1a620.snappy.parquet',
  's3://japanese-stocks/jquants-api/price/part

# List

In [3]:
# TOPIXのヒストリカルデータ
df = pd.read_pickle("../colab_data/list_20080101_20240407.pkl")
df["Date"] = pd.to_datetime(df["Date"])

# symbol represent a group of data for given data columns
df["symbol"] = "jquants_api"

# timestamp should be UTC timezone but without tz info
df["dt"] = df["Date"].dt.tz_localize(None)

# partition_dt must be date, data will be updated partition by partition with use of this column.
# Every time, you have to upload all the data for a given partition_dt, otherwise older will be gone.
df["partition_dt"] = df["dt"].dt.date.map(lambda x: x.replace(day=1))

print(df.shape)
df.head()

(2877911, 13)


,Date,Code,CompanyNameEnglish,Sector17Code,Sector33Code,ScaleCategory,MarketCode,MarketCodeName,MarginCode,MarginCodeName,symbol,dt,partition_dt
0,2008-05-07,13010,"KYOKUYO CO.,LTD.",1,0050,TOPIX Small 2,0101,東証一部,2,貸借,jquants_api,2008-05-07,2008-05-01
1,2008-05-07,13080,Nikko Exchange Traded Index Fund TOPIX,99,9999,-,0109,その他,2,貸借,jquants_api,2008-05-07,2008-05-01
2,2008-05-07,13100,Daiwa ETF-TOPIX Core30,99,9999,-,0109,その他,2,貸借,jquants_api,2008-05-07,2008-05-01
3,2008-05-07,13110,TOPIX Core 30 Exchange Traded Fund,99,9999,-,0109,その他,2,貸借,jquants_api,2008-05-07,2008-05-01
4,2008-05-07,13130,KODEX200 Exchange Traded Fund,99,9999,-,0109,その他,2,貸借,jquants_api,2008-05-07,2008-05-01


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2877911 entries, 0 to 2877910
Data columns (total 13 columns):
 #   Column              Dtype         
---  ------              -----         
 0   Date                datetime64[ns]
 1   Code                object        
 2   CompanyNameEnglish  object        
 3   Sector17Code        object        
 4   Sector33Code        object        
 5   ScaleCategory       object        
 6   MarketCode          object        
 7   MarketCodeName      object        
 8   MarginCode          object        
 9   MarginCodeName      object        
 10  symbol              object        
 11  dt                  datetime64[ns]
 12  partition_dt        object        
dtypes: datetime64[ns](2), object(11)
memory usage: 285.4+ MB


In [5]:
tsdb.upload(table_name="list", df=df)

{'paths': ['s3://japanese-stocks/jquants-api/list/partition_dt=2022-07-01/symbol=jquants_api/deb17a379ab14692b30eb3ab3640c158.snappy.parquet',
  's3://japanese-stocks/jquants-api/list/partition_dt=2019-07-01/symbol=jquants_api/deb17a379ab14692b30eb3ab3640c158.snappy.parquet',
  's3://japanese-stocks/jquants-api/list/partition_dt=2022-03-01/symbol=jquants_api/deb17a379ab14692b30eb3ab3640c158.snappy.parquet',
  's3://japanese-stocks/jquants-api/list/partition_dt=2017-12-01/symbol=jquants_api/deb17a379ab14692b30eb3ab3640c158.snappy.parquet',
  's3://japanese-stocks/jquants-api/list/partition_dt=2023-05-01/symbol=jquants_api/deb17a379ab14692b30eb3ab3640c158.snappy.parquet',
  's3://japanese-stocks/jquants-api/list/partition_dt=2021-06-01/symbol=jquants_api/deb17a379ab14692b30eb3ab3640c158.snappy.parquet',
  's3://japanese-stocks/jquants-api/list/partition_dt=2012-12-01/symbol=jquants_api/deb17a379ab14692b30eb3ab3640c158.snappy.parquet',
  's3://japanese-stocks/jquants-api/list/partition_dt